In [7]:
pip install yfinance gym pandas numpy statsmodels


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: C:\Users\saura\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [21]:
import pandas as pd
import numpy as np
import gym
from statsmodels.tsa.arima.model import ARIMA
import yfinance as yf

# Define the custom trading environment
class StockTradingEnv(gym.Env):
    def __init__(self, df):
        super(StockTradingEnv, self).__init__()
        self.df = df
        self.action_space = gym.spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(5,), dtype=np.float32)
        self.reset()

    def reset(self):
        self.current_step = 0
        self.cash = 10000
        self.shares_held = 0
        return self._next_observation()

    def _next_observation(self):
        obs = np.array([
            self.df.iloc[self.current_step]['Close'],
            self.cash,
            self.shares_held,
            self.df.iloc[self.current_step]['MACD'],
            self.df.iloc[self.current_step]['RSI']
        ])
        return obs

    def step(self, action):
        current_price = self.df.iloc[self.current_step]['Close']
        action = action[0]

        if action > 0:  # buy
            n_shares = self.cash * action / current_price
            self.cash -= n_shares * current_price
            self.shares_held += n_shares
        elif action < 0:  # sell
            n_shares = -self.shares_held * action
            self.cash += n_shares * current_price
            self.shares_held -= n_shares

        self.current_step += 1
        done = self.current_step >= len(self.df) - 1
        reward = self.cash + self.shares_held * current_price

        obs = self._next_observation()
        return obs, reward, done, {}

    def render(self, mode='human', close=False):
        pass




In [22]:

# Download stock data from Yahoo Finance
ticker = "^NSEI" 
start_date = "2023-08-10"
end_date = "2024-08-10"
df = yf.download(ticker, start=start_date, end=end_date)


[*********************100%%**********************]  1 of 1 completed


In [23]:
 #Add technical indicators (MACD and RSI)
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    short_ema = data['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = data['Close'].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    return macd, signal

def calculate_rsi(data, window=14):
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['MACD'], df['Signal'] = calculate_macd(df)
df['RSI'] = calculate_rsi(df)
df.dropna(inplace=True)  

# Set frequency for date index
df.index = pd.to_datetime(df.index)
df = df.asfreq('D')  # Assuming daily frequency

In [24]:
# Split the data
train_df = df[:int(0.6*len(df))]
validate_df = df[int(0.6*len(df)):int(0.8*len(df))]
test_df = df[int(0.8*len(df)):]

In [25]:
# Fit ARIMA model on the training data
train_close = train_df['Close']
model = ARIMA(train_close, order=(5, 1, 0))  # ARIMA(p, d, q)
model_fit = model.fit()

# Predict on the validation data
validate_predictions = model_fit.forecast(steps=len(validate_df))
validate_predictions_series = pd.Series(validate_predictions, index=validate_df.index)

# Calculate returns
validate_returns = validate_df['Close'].pct_change().dropna()
predicted_returns = validate_predictions_series.pct_change().dropna()

# Calculate Sharpe Ratio manually
def calculate_sharpe_ratio(returns, risk_free_rate=0):
    mean_return = returns.mean()
    return_std = returns.std()
    if return_std == 0:
        return np.nan  # Avoid division by zero
    sharpe_ratio = (mean_return - risk_free_rate) / return_std
    return sharpe_ratio

sharpe_ratio_validate = calculate_sharpe_ratio(validate_returns)
print(f"Validation Sharpe Ratio: {sharpe_ratio_validate}")


Validation Sharpe Ratio: 0.0707660197834547


C:\Users\saura\AppData\Local\Temp\ipykernel_8040\1436750106.py:11: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  validate_returns = validate_df['Close'].pct_change().dropna()


In [26]:
# Predict on the test data
test_predictions = model_fit.forecast(steps=len(test_df))
test_predictions_series = pd.Series(test_predictions, index=test_df.index)

# Calculate Sharpe Ratio for test set
test_returns = test_df['Close'].pct_change().dropna()
predicted_test_returns = test_predictions_series.pct_change().dropna()
sharpe_ratio_test = calculate_sharpe_ratio(test_returns)
print(f"Test Sharpe Ratio: {sharpe_ratio_test}")

# Final Portfolio Value
test_env = StockTradingEnv(test_df)
test_env.reset()

# Simulate holding strategy (action = [0] for holding)
actions = [[0]] * len(test_df)  # Hold all the time
for action in actions:
    obs, reward, done, info = test_env.step(action)
    if done:
        break

portfolio_value = test_env.cash + test_env.shares_held * test_df.iloc[test_env.current_step]['Close']
print(f"Final Portfolio Value: {portfolio_value}")


Test Sharpe Ratio: 0.07141203823868351
Final Portfolio Value: 10000.0


C:\Users\saura\AppData\Local\Temp\ipykernel_8040\930421905.py:6: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  test_returns = test_df['Close'].pct_change().dropna()
C:\Users\saura\AppData\Local\Temp\ipykernel_8040\930421905.py:7: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  predicted_test_returns = test_predictions_series.pct_change().dropna()


In [27]:
returns = test_env.df['Close'].pct_change().dropna()

C:\Users\saura\AppData\Local\Temp\ipykernel_8040\795205414.py:1: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = test_env.df['Close'].pct_change().dropna()


In [28]:
print(returns)

Date
2024-06-04   -0.059294
2024-06-05    0.033624
2024-06-06    0.008888
2024-06-07    0.020540
2024-06-08    0.000000
                ...   
2024-08-05   -0.026786
2024-08-06   -0.002621
2024-08-07    0.012710
2024-08-08   -0.007429
2024-08-09    0.010387
Freq: D, Name: Close, Length: 67, dtype: float64


In [29]:
print(f"Sharpe Ratio: {sharpe_ratio_test}")

Sharpe Ratio: 0.07141203823868351
